In [26]:
import numpy as np

def comp_dist(x1, x2):
    # Dimensional parameters
    nx1, nx2 = x1.shape[1], x2.shape[1]

    # Compute distance matrix
    X1 = np.tile(np.sum(x1 ** 2, axis=0), (nx2, 1))
    X2 = np.tile(np.sum(x2 ** 2, axis=0), (nx1, 1))
    D = X1.T + X2 - 2 * np.dot(x1.T, x2)

    # Return
    return D

In [27]:
def comp_median(x):
    # Dimension parameters
    d, n = x.shape

    # Calculate median
    G = np.tile(np.sum(x ** 2, axis=0), (n, 1))
    D = G - 2 * np.dot(x.T, x) + G.T
    np.fill_diagonal(D, 0)  # Ensures the diagonal is zero, similar to setting upper triangular part to 0 in R

    # Return
    return np.sqrt(0.5 * np.median(D[D > 0]))


In [28]:
def gaussian_kernel(distance_matrix, sigma):
    return np.exp(-distance_matrix / (2 * sigma ** 2))

In [29]:
def sigma_lambda_grid_search(Nnu, Nde, k, dist_nu, dist_de, sigma_list, lambda_list, alpha, n_folds):
    n_sigma = len(sigma_list)
    n_lambda = len(lambda_list)
    scores = np.zeros((n_sigma, n_lambda))

    for s_idx, sigma in enumerate(sigma_list):
        ker_nu = gaussian_kernel(dist_nu, sigma)
        ker_de = gaussian_kernel(dist_de, sigma)

        for l_idx, lambda_ in enumerate(lambda_list):
            folds_nu = np.random.choice(n_folds, Nnu)
            folds_de = np.random.choice(n_folds, Nde)

            obj_score_sum = 0
            for f in range(n_folds):
                nu_train = ker_nu[folds_nu != f].T
                de_train = ker_de[folds_de != f].T
                nu_test = ker_nu[folds_nu == f].T
                de_test = ker_de[folds_de == f].T

                H_k = ((1 - alpha) / de_train.shape[1]) * np.dot(de_train, de_train.T) + \
                      (alpha / nu_train.shape[1]) * np.dot(nu_train, nu_train.T)
                h_k = np.mean(nu_train, axis=1)

                theta_hat = np.linalg.solve(H_k + np.eye(k) * lambda_, h_k)

                J = (alpha / 2) * np.mean((np.dot(theta_hat.T, nu_test)) ** 2) + \
                    ((1 - alpha) / 2) * np.mean((np.dot(theta_hat.T, de_test)) ** 2) - \
                    np.mean(np.dot(theta_hat.T, nu_test))

                obj_score_sum += J

            scores[s_idx, l_idx] = obj_score_sum / n_folds

    opt_indices = np.unravel_index(scores.argmin(), scores.shape)
    opt_sigma = sigma_list[opt_indices[0]]
    opt_lambda = lambda_list[opt_indices[1]]

    return {'opt_sigma': opt_sigma, 'opt_lambda': opt_lambda}


In [30]:
def sliding_window(X, window_size):
    # Dimension parameters
    dim_ts, N_points = X.shape
    N_subsequences = N_points - window_size + 1

    # Create template array to fill in
    W = np.zeros((dim_ts * window_size, N_subsequences))

    # Fill in the result
    for i in range(N_subsequences):
        W[:, i] = X[:, i:i + window_size].flatten('F')  # 'F' means Fortran-style order

    # Return
    return W

In [31]:
import numpy as np

def RelULSIF(Xnu, Xde, Xce=None, sigma=None, lambda_=None, alpha=0.01, k=100, n_folds=5):
    # Ensure Xnu and Xde are numpy arrays
    Xnu, Xde = np.atleast_2d(Xnu), np.atleast_2d(Xde)
    
    # Check on alpha
    if not 0 <= alpha < 1:
        raise ValueError("Parameter alpha must be in [0, 1).")
    
    # Check on sigma
    if sigma is not None:
        sigma = np.atleast_1d(sigma)
        if np.any(sigma <= 0):
            raise ValueError("Values for sigma must be positive.")

    # Check on lambda
    if lambda_ is not None:
        lambda_ = np.atleast_1d(lambda_)
        if np.any(lambda_ < 0):
            raise ValueError("Values for lambda must be non-negative.")
    
    # Check on k
    if not isinstance(k, int) or k <= 0:
        raise ValueError("Parameter k must be a positive integer.")
    
    # Check on n_folds
    if not isinstance(n_folds, int) or n_folds <= 0:
        raise ValueError("Parameter n_folds must be a positive integer.")
    
    # Dimension checks
    if Xnu.shape[0] != Xde.shape[0]:
        raise ValueError("Number of rows must match between Xnu and Xde.")
    
    # Ensure k does not exceed dimensions
    k = min(k, Xnu.shape[0])
    
    # Initialize centers for Gaussian kernel if NULL
    if Xce is None:
        idx = np.random.choice(Xnu.shape[1], size=k, replace=True)
        Xce = Xnu[:, idx]
    
    # Compute distance matrices
    dist_nu = comp_dist(Xnu, Xce)
    dist_de = comp_dist(Xde, Xce)
    
    # Cross-validation for optimal sigma and lambda if needed
    if sigma is None or lambda_ is None or len(sigma) > 1 or len(lambda_) > 1:
        if sigma is None:
            med = comp_median(np.hstack((Xde, Xnu)))
            sigma_search_vec = med * np.linspace(0.6, 1.4, num=5)
        else:
            sigma_search_vec = sigma
        
        if lambda_ is None:
            lambda_search_vec = 10.0 ** np.arange(-3, 2)
        else:
            lambda_search_vec = lambda_
        
        cv_out = sigma_lambda_grid_search(Xnu.shape[1], Xde.shape[1], k, dist_nu, dist_de,
                                          sigma_search_vec, lambda_search_vec, alpha, n_folds)
        opt_sigma, opt_lambda = cv_out['opt_sigma'], cv_out['opt_lambda']
    else:
        opt_sigma, opt_lambda = sigma, lambda_
    
    # Compute kernel matrices
    ker_Xnu = gaussian_kernel(dist_nu.T, opt_sigma)
    ker_Xde = gaussian_kernel(dist_de.T, opt_sigma)
    
    # Compute H, h, and solve for theta
    H = ((1 - alpha) / Xde.shape[1]) * np.dot(ker_Xde, ker_Xde.T) + \
        (alpha / Xnu.shape[1]) * np.dot(ker_Xnu, ker_Xnu.T)
    h = np.mean(ker_Xnu, axis=1)
    theta = np.linalg.solve(H + np.eye(k) * opt_lambda, h)
    
    # Compute g_nu, g_de, and rPE
    g_nu = np.dot(theta.T, ker_Xnu)
    g_de = np.dot(theta.T, ker_Xde)
    rPE = np.mean(g_nu) - 0.5 * ((alpha * np.mean(g_nu ** 2)) + (1 - alpha) * np.mean(g_de ** 2)) - 0.5
    
    return {'opt_sigma': opt_sigma, 'opt_lambda': opt_lambda, 'rPE': rPE}

In [32]:
import matplotlib.pyplot as plt
import numpy as np

def ts_plot(ts, step, scores, change_points):
    ts_dim, n_time_points = ts.shape
    time_range = np.arange(1, n_time_points + 1)
    
    fig, axes = plt.subplots(ts_dim + 1, 1, figsize=(12, 3 * (ts_dim + 1)), sharex=True)
    
    for s in range(ts_dim):
        ax = axes[s]
        ax.plot(time_range, ts[s, :], label=f'Dimension {s+1}')
        ax.set_ylabel(f'y{s+1}')
        for cp in change_points:
            ax.axvspan(cp, cp + 1, color='#DD6666', alpha=0.5)
    
    # Plot scores
    scores_ax = axes[-1]
    scores_ax.plot(np.arange(len(scores)) + step, scores, color='red', label='rPE Scores')
    scores_ax.set_xlabel('Time')
    scores_ax.set_ylabel('rPE')
    scores_ax.legend()

    plt.tight_layout()
    plt.show()

In [33]:
import numpy as np

def ts_detect(ts, window_size=5, step=None, alpha=0.05, k=100, n_folds=5, thresh=0.9, make_plot=False):
    ts = np.atleast_2d(ts)
    
    # Dimensional parameters
    dim_ts, N_time_points = ts.shape
    print(dim_ts)

    # Window size check
    if window_size <= 0 or not isinstance(window_size, int):
        raise ValueError("Parameter window_size must be a positive integer.")
    elif window_size > (N_time_points - 1):
        raise ValueError("Parameter window_size too large. Try reducing it.")
    
    # Step check
    if step is None:
        step = int(0.1 * N_time_points)
    elif step <= 0 or not isinstance(step, int):
        raise ValueError("Parameter step must be a positive integer.")
    elif step > (N_time_points - window_size):
        raise ValueError("Parameter step is too large. Try reducing it.")
    
    # Alpha check
    if alpha < 0 or alpha >= 1:
        raise ValueError("Parameter alpha must be in [0, 1).")

    # k check
    k = min(k, dim_ts)

    # n_folds check
    if n_folds <= 0 or not isinstance(n_folds, int):
        raise ValueError("Parameter n_folds must be a positive integer.")
    elif n_folds > N_time_points:
        raise ValueError("Parameter n_folds exceeds number of points in the input time series.")

    # Thresh check
    if thresh <= 0 or thresh >= 1:
        raise ValueError("Parameter thresh should be a scalar in (0, 1).")
    
    # Construct sliding window
    sw = sliding_window(ts, window_size)
    n_samples = sw.shape[1]
    
    # Compute change-point scores
    scores = []
    t = step + 1
    while t + step - 1 <= n_samples:
        y = sw[:, (t - step):(step + t - 1)]
        y_nu = y[:, :step]
        y_de = y[:, step:]
        
        out = RelULSIF(y_nu, y_de, alpha=alpha, k=k, n_folds=n_folds)
        scores.append(out['rPE'])
        t += 1
    
    scores = np.array(scores)
    change_points = np.where(scores > thresh * np.max(scores))[0] + step
    
    # Make plot, if asked for
    if make_plot:
        ts_plot(ts, step, scores, change_points)

    return {'step': step, 'scores': scores, 'change_points': change_points}

In [35]:
import numpy as np
import pandas as pd

series = np.loadtxt("dros067.txt", delimiter=' ')

# Transpose the series to get dimensions as rows and time points as columns
seriesT = series.T

# Now you can call ts_detect on this transposed series
d = ts_detect(seriesT, window_size=5, step=10, make_plot=True)

11


ValueError: Integers to negative integer powers are not allowed.

In [ ]:
# Generate synthetic series
series = np.concatenate([
    np.random.normal(0, 0.3, 50),
    np.random.normal(8, 1, 25),
    np.random.normal(3, 0.6, 75),
    np.random.normal(1, 0.8, 25),
    np.random.normal(-5, 1.5, 100),
    np.random.normal(-5, 0.2, 100),
    np.random.normal(-2.5, 0.4, 50),
    np.random.normal(2, 1.2, 50)
])

# Detect change points in the synthetic series
d = ts_detect(series.reshape(1, -1), window_size=3, step=10, make_plot=True)


ValueError: Integers to negative integer powers are not allowed.